<a href="https://colab.research.google.com/github/rajan-sharma-in/GDG/blob/main/AI_Buddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Demo installs:
# - Pin transformers to avoid: ImportError gather_state_dict_for_save
# - Pin pillow to avoid random PIL typing issues

%pip -q install -U faiss-cpu gradio pymupdf google-genai

%pip -q install -U \
  "transformers==4.45.2" \
  "sentence-transformers==3.1.1" \
  "tokenizers==0.20.1" \
  "safetensors==0.4.5" \
  "huggingface-hub==0.26.2"

%pip -q install -U "pillow==10.4.0"


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.1 requires accelerate>=0.21.0, which is not installed.
transformers 4.45.2 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 1.4.1 which is incompatible.
tokenizers 0.20.1 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 1.4.1 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2026.2.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.1 requires accelerate>=0.21.0, which is not installed.
gradio 6.6.0 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.26.2 which is incompatible.
diffusers 0.36.0 requires huggingface-hub<2.0,>=0.34.0, but you ha

In [ ]:
#
import os, re, json
from pathlib import Path

import numpy as np
import pandas as pd
import fitz  # PyMuPDF
import faiss
from sentence_transformers import SentenceTransformer


ImportError: cannot import name 'gather_state_dict_for_save' from 'transformers.integrations.tensor_parallel' (/usr/local/lib/python3.12/dist-packages/transformers/integrations/tensor_parallel.py)

In [ ]:
USE_DRIVE = True

if USE_DRIVE:
    from google.colab import drive
    drive.mount("/content/drive")
    PDF_DIR = Path("/content/drive/MyDrive/Colab_PDFs")
    CACHE_DIR = Path("/content/drive/MyDrive/Colab_PDFs/index_cache")
else:
    PDF_DIR = Path("/content/pdfs")
    CACHE_DIR = Path("/content/index_cache")

PDF_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

def list_pdfs():
    pdfs = sorted(PDF_DIR.glob("*.pdf"))
    print(f"PDF folder: {PDF_DIR}")
    print(f"Found {len(pdfs)} PDF(s).")
    for p in pdfs[:30]:
        print(" -", p.name)
    if len(pdfs) > 30:
        print(" ... (showing first 30)")
    return pdfs

pdf_paths = list_pdfs()


ImportError: cannot import name '_Ink' from 'PIL._typing' (/usr/local/lib/python3.12/dist-packages/PIL/_typing.py)

In [ ]:
# If you use Google Drive for storing many big books:
USE_DRIVE = True

if USE_DRIVE:
    from google.colab import drive
    drive.mount("/content/drive")
    PDF_DIR = Path("/content/drive/MyDrive/Colab_PDFs")
    CACHE_DIR = Path("/content/drive/MyDrive/Colab_PDFs/index_cache_v2")
else:
    PDF_DIR = Path("/content/pdfs")
    CACHE_DIR = Path("/content/index_cache_v2")

PDF_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

def list_pdfs():
    pdfs = sorted(PDF_DIR.glob("*.pdf"))
    print("PDF_DIR:", PDF_DIR)
    print("Found PDFs:", len(pdfs))
    for p in pdfs[:30]:
        print(" -", p.name)
    if len(pdfs) > 30:
        print(" ... (showing first 30)")
    return pdfs

pdf_paths = list_pdfs()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PDF_DIR: /content/drive/MyDrive/Colab_PDFs
Found PDFs: 1
 - 16_EBOOK-7th_ed_software_engineering_a_practitioners_approach_by_roger_s._pressman_.pdf


In [ ]:
# Cell 5 — Preprocess + Chunking
# - Cleans whitespace
# - Splits into overlapping character chunks
# - Extracts chapter-like headings for rule-based queries
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 180
MAX_CHUNKS = 15000  # guardrail for very large corpora

def clean_text(t: str) -> str:
    return re.sub(r"\s+", " ", t).strip()

def make_chunks(df_pages, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    chunks = []
    for _, row in df_pages.iterrows():
        text = clean_text(row["text"])
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            chunks.append({
                "chunk_id": f"{row.file_name}__p{row.page_number}__{start}",
                "file_name": row.file_name,
                "page_number": row.page_number,
                "chunk_text": chunk_text,
                "low_text": row.low_text,
            })
            start = end - overlap
    return pd.DataFrame(chunks)

def extract_chapter_candidates(df_pages):
    heads = []
    pattern = re.compile(r"\bchap(?:ter)?\b\s*[0-9ivxlcdm]*", re.IGNORECASE)
    for _, row in df_pages.iterrows():
        for line in row["text"].splitlines():
            clean = line.strip()
            if not clean or len(clean) > 90:
                continue
            if pattern.search(clean) or ("chapter" in clean.lower() and len(clean.split()) <= 8):
                heads.append({
                    "file_name": row.file_name,
                    "page_number": row.page_number,
                    "heading": clean_text(clean),
                })
    df = pd.DataFrame(heads)
    if len(df):
        df = df.drop_duplicates(subset=["file_name", "heading"], keep="first").sort_values(["file_name", "page_number"])
    return df

if len(pages_df) == 0:
    chunks_df = pd.DataFrame(columns=["chunk_id","file_name","page_number","chunk_text","low_text"])
    chapter_df = pd.DataFrame(columns=["file_name","page_number","heading"])
else:
    chunks_df = make_chunks(pages_df)
    chapter_df = extract_chapter_candidates(pages_df)
    if len(chunks_df) > MAX_CHUNKS:
        print(f"Too many chunks ({len(chunks_df)}). Consider fewer PDFs or larger chunk_size.")
    print(f"Chunks created: {len(chunks_df)} | Avg chars/chunk: {chunks_df['chunk_text'].str.len().mean():.1f}")
    print(f"Chapter-like headings detected: {len(chapter_df)}")

chunks_df.head()

Chunks created: 3502 | Avg chars/chunk: 846.2
Chapter-like headings detected: 494


,chunk_id,file_name,page_number,chunk_text,low_text
0,16_EBOOK-7th_ed_software_engineering_a_practit...,16_EBOOK-7th_ed_software_engineering_a_practit...,1,Software Engineering A Practitioner’s Approach...,False
1,16_EBOOK-7th_ed_software_engineering_a_practit...,16_EBOOK-7th_ed_software_engineering_a_practit...,2,Software Engineering A P R A C T I T I O N E R...,False
2,16_EBOOK-7th_ed_software_engineering_a_practit...,16_EBOOK-7th_ed_software_engineering_a_practit...,4,Software Engineering A P R A C T I T I O N E R...,False
3,16_EBOOK-7th_ed_software_engineering_a_practit...,16_EBOOK-7th_ed_software_engineering_a_practit...,5,SOFTWARE ENGINEERING: A PRACTITIONER’S APPROAC...,False
4,16_EBOOK-7th_ed_software_engineering_a_practit...,16_EBOOK-7th_ed_software_engineering_a_practit...,5,0 DOC/DOC 0 9 ISBN 978–0–07–337597–7 MHID 0–07...,False


In [ ]:
# Cell 6 — Embeddings + FAISS Index
# - Loads MiniLM model
# - Builds/loads FAISS cosine index
# - Caches index + metadata to /content/index_cache
MODEL_NAME = "all-MiniLM-L6-v2"
INDEX_PATH = CACHE_DIR / "faiss.index"
META_PATH = CACHE_DIR / "chunks.parquet"
EMB_PATH = CACHE_DIR / "embeddings.npy"
MANIFEST_PATH = CACHE_DIR / "manifest.json"

model = SentenceTransformer(MODEL_NAME)

def current_manifest(pdf_paths):
    return {
        str(p.name): {"mtime": p.stat().st_mtime, "size": p.stat().st_size}
        for p in pdf_paths
    }

def load_cache(pdf_paths):
    if not (INDEX_PATH.exists() and META_PATH.exists() and EMB_PATH.exists() and MANIFEST_PATH.exists()):
        return None
    saved = json.load(open(MANIFEST_PATH))
    if saved != current_manifest(pdf_paths):
        return None
    try:
        df = pd.read_parquet(META_PATH)
        emb = np.load(EMB_PATH)
        idx = faiss.read_index(str(INDEX_PATH))
        print("Loaded cached index.")
        return df, emb, idx
    except Exception as e:
        print("Cache load failed:", e)
        return None

def save_cache(df, emb, idx, pdf_paths):
    df.to_parquet(META_PATH, index=False)
    np.save(EMB_PATH, emb)
    faiss.write_index(idx, str(INDEX_PATH))
    json.dump(current_manifest(pdf_paths), open(MANIFEST_PATH, "w"))
    print("Cache saved to /content/index_cache.")

def build_index(pdf_paths, force=False):
    if len(pdf_paths) == 0:
        print("No PDFs to index.")
        return None, None, None
    if not force:
        cached = load_cache(pdf_paths)
        if cached:
            return cached

    print("Encoding chunks...")
    if len(chunks_df) == 0:
        print("No chunks to index yet.")
        return None, None, None

    emb = model.encode(
        chunks_df["chunk_text"].tolist(),
        convert_to_numpy=True,
        show_progress_bar=True
    ).astype("float32")
    faiss.normalize_L2(emb)

    dim = emb.shape[1]
    idx = faiss.IndexFlatIP(dim)
    idx.add(emb)
    save_cache(chunks_df, emb, idx, pdf_paths)
    return chunks_df, emb, idx

chunks_df, emb_matrix, index = build_index(pdf_paths, force=False)
if index:
    print(f"FAISS ready | dim: {emb_matrix.shape[1]} | vectors: {index.ntotal}")
    print(f"Indexed PDFs: {len(pdf_paths)}, Total pages: {len(pages_df)}, Total chunks: {len(chunks_df)}")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loaded cached index.
FAISS ready | dim: 384 | vectors: 3502
Indexed PDFs: 1, Total pages: 930, Total chunks: 3502


In [ ]:
# Cell 7 — Static Retrieval Demo
# - Retrieves top-K chunks
# - Detects chapter/page-count/list-doc/author queries for routing

def confidence_label(score: float) -> str:
    if score >= 0.65: return "High"
    if score >= 0.40: return "Medium"
    return "Low"

def preview(text, n=240):
    return (text[:n] + " ...") if len(text) > n else text

def is_chapter_query(q: str) -> bool:
    return bool(re.search(r"chapter( names?| list| titles?)|list .*chapters|all chapters", q, re.IGNORECASE))

def is_pagecount_query(q: str) -> bool:
    return bool(re.search(r"how many pages|page count|total pages|number of pages", q, re.IGNORECASE))

def is_list_docs_query(q: str) -> bool:
    return bool(re.search(r"list (pdfs|books|documents)|what (files|pdfs)|which books", q, re.IGNORECASE))

def is_author_query(q: str) -> bool:
    return bool(re.search(r"who is the author|author of|who wrote|written by", q, re.IGNORECASE))

def retrieve_chunks(query: str, k: int = 3, doc_filter: str = "All"):
    if index is None:
        return {"error": "Index not built yet."}
    base_df = chunks_df if doc_filter == "All" else chunks_df[chunks_df.file_name == doc_filter]
    if len(base_df) == 0:
        return {"error": f"No chunks for '{doc_filter}'. Rebuild index or pick All."}

    q = clean_text(query)
    q_emb = model.encode([q], convert_to_numpy=True, show_progress_bar=False).astype("float32")
    faiss.normalize_L2(q_emb)

    if doc_filter == "All":
        k_eff = min(k, len(base_df))
        scores, idxs = index.search(q_emb, k_eff)
        rows = base_df.iloc[idxs[0]]
    else:
        sub = base_df.reset_index(drop=True)
        sub_emb = emb_matrix[sub.index]
        sub_index = faiss.IndexFlatIP(sub_emb.shape[1])
        sub_index.add(sub_emb)
        k_eff = min(k, len(sub))
        scores, idxs = sub_index.search(q_emb, k_eff)
        rows = sub.iloc[idxs[0]]

    items = []
    for r, row in enumerate(rows.itertuples()):
        score = float(scores[0][r])
        items.append({
            "score": score,
            "confidence": confidence_label(score),
            "file_name": row.file_name,
            "page_number": int(row.page_number),
            "chunk_text": row.chunk_text,
            "preview": preview(row.chunk_text),
        })
    return {"error": None, "items": items, "top_score": float(scores[0][0]) if len(items) else 0.0}

# quick smoke test
demo = retrieve_chunks("Quality Concept", k=3)
demo["items"][:1] if demo["error"] is None else demo


[{'score': 0.7885301113128662,
  'confidence': 'High',
  'file_name': '16_EBOOK-7th_ed_software_engineering_a_practitioners_approach_by_roger_s._pressman_.pdf',
  'page_number': 440,
  'chunk_text': ' CHAPTER 14 QUALITY CONCEPTS 411',
  'preview': ' CHAPTER 14 QUALITY CONCEPTS 411'}]

In [ ]:
# Cell 8 — Pipeline functions (retrieve + rag + metadata routes)
# - Rebuilds index
# - Routes metadata queries (page count, list docs, chapters, author) before embeddings

def rebuild_index(force=False):
    global pdf_paths, pages_df, chunks_df, emb_matrix, index, chapter_df, doc_stats
    pdf_paths = list_pdfs()
    if len(pdf_paths) == 0:
        return "No PDFs found. Upload files first."
    pages_df, doc_stats = extract_pdfs(pdf_paths)
    chunks_df = make_chunks(pages_df)
    chapter_df = extract_chapter_candidates(pages_df)
    if len(chunks_df) > MAX_CHUNKS:
        return f"Too many chunks ({len(chunks_df)}). Use fewer PDFs."
    chunks_df, emb_matrix, index = build_index(pdf_paths, force=force)
    if index:
        return f"Rebuilt index. PDFs: {len(pdf_paths)}, pages: {len(pages_df)}, chunks: {len(chunks_df)}, chapters: {len(chapter_df)}"
    return "Index rebuild failed."

def ensure_index_ready():
    if 'index' not in globals() or index is None or 'pages_df' not in globals() or pages_df is None:
        return rebuild_index(force=True)
    return "Index ready (cache loaded)."

def sync_pdfs():
    # Smart sync: reuse cache when unchanged; rebuild when new/modified files
    return rebuild_index(force=False)

def chapter_answer(doc_filter="All"):
    if chapter_df is None or len(chapter_df) == 0:
        return "No chapter-like headings found.", "", ""
    df = chapter_df if doc_filter == "All" else chapter_df[chapter_df.file_name == doc_filter]
    if len(df) == 0:
        return f"No chapter headings in {doc_filter}.", "", ""
    lines = []
    for row in df.itertuples():
        cite = f"[{row.file_name} p.{row.page_number}]"
        lines.append(f"{cite} {row.heading}")
    answer_text = "Chapter-like headings I found:\n" + "\n".join(lines)
    transparency = "\n".join(lines)
    prompt = "Rule-based chapter heading extraction (no LLM)."
    return answer_text, "Metadata", transparency

def pagecount_answer(doc_filter="All"):
    if doc_stats is None or len(doc_stats) == 0:
        return "No PDFs indexed yet.", "", ""
    df = doc_stats if doc_filter == "All" else doc_stats[doc_stats.file_name == doc_filter]
    if len(df) == 0:
        return f"No stats for {doc_filter}.", "", ""
    lines = []
    for row in df.itertuples():
        lines.append(f"{row.file_name}: {row.total_pages} pages (low-text {row.low_text_pages})")
    ans = "\n".join(lines)
    return ans, "Metadata", ans

def listdocs_answer():
    if doc_stats is None or len(doc_stats) == 0:
        return "No PDFs indexed yet.", "", ""
    lines = [f"{row.file_name} ({row.total_pages} pages)" for row in doc_stats.itertuples()]
    ans = "Available PDFs:\n" + "\n".join(lines)
    return ans, "Metadata", ans

def author_answer(doc_filter="All"):
    if doc_stats is None or len(doc_stats) == 0:
        return "No PDFs indexed yet.", "", ""
    df = doc_stats if doc_filter == "All" else doc_stats[doc_stats.file_name == doc_filter]
    if len(df) == 0:
        return f"No author info for {doc_filter}.", "", ""
    lines = []
    for row in df.itertuples():
        lines.append(f"{row.file_name}: {row.author} (Title: {row.title})")
    ans = "\n".join(lines)
    return ans, "Metadata", ans

def rag_answer(question, items, use_gemini=False):
    context = "\n\n".join([item["chunk_text"] for item in items])
    if not use_gemini:
        answer = "I found the following information in the documents:\n\n" + context
        prompt = "Direct embedding retrieval (no LLM)."
        return answer, prompt
    else:
        # Attempt to use Gemini model if it's available
        if 'genai' in globals() and 'gemini_model' in globals():
            try:
                gemini_prompt = f"""
                Based ONLY on the following information, provide a helpful and conversational response to the query.
                Do NOT include any information not present in the provided context.

                Query: "{question}"

                Information from documents:
                {context}

                Please synthesize this information and provide a clear answer.
                """
                response = gemini_model.generate_content(gemini_prompt)
                answer = response.text
                prompt = "Gemini RAG with context."
                return answer, prompt
            except Exception as e:
                answer = f"Error with Gemini: {e}. Returning raw chunks: \n\n" + context
                prompt = "Gemini error, falling back to direct chunks."
                return answer, prompt
        else:
            answer = "Gemini not configured or initialized. Returning raw chunks: \n\n" + context
            prompt = "Gemini not available, falling back to direct chunks."
            return answer, prompt

def pipeline(question, top_k=3, doc_filter="All", use_gemini=False):
    if not question.strip():
        return "Please enter a question.", "", "", ""

    # ensure index/data available (lazy init)
    ready_msg = ensure_index_ready()

    if is_list_docs_query(question):
        ans, conf, trans = listdocs_answer()
        return ans, conf, trans, "List docs route"
    if is_pagecount_query(question):
        ans, conf, trans = pagecount_answer(doc_filter)
        return ans, conf, trans, "Page-count route"
    if is_author_query(question):
        ans, conf, trans = author_answer(doc_filter)
        return ans, conf, trans, "Author route"
    if is_chapter_query(question):
        ans, conf, trans = chapter_answer(doc_filter)
        return ans, conf, trans, "Chapter route"

    res = retrieve_chunks(question, k=int(top_k), doc_filter=doc_filter)
    if res["error"]:
        return res["error"], "", "", ""
    items = res["items"]
    top = items[0]
    conf = f"{confidence_label(top['score'])} (top score {top['score']:.3f})"
    answer_text, prompt = rag_answer(question, items, use_gemini=use_gemini)
    lines = []
    for it in items:
        cite = f"[{it['file_name']} p.{it['page_number']}]"
        lines.append(f"Score: {it['score']:.3f} | {it['confidence']} | {cite}\n{it['preview']}")
    transparency = "\n\n".join(lines)
    return answer_text, conf, transparency, prompt

print("Call pipeline(question, top_k, doc_filter, use_gemini) or sync_pdfs() / rebuild_index(force=True).")

Call pipeline(question, top_k, doc_filter, use_gemini) or sync_pdfs() / rebuild_index(force=True).


In [ ]:
# Cell X — Embedding Search Result Demo
# This cell demonstrates retrieving embedding search results for a specific question.

print("Running embedding search for the question...")
question = "what is software testing strategies"

# Check if pipeline is defined, otherwise provide instructions
if 'pipeline' not in globals():
    print("Error: The 'pipeline' function is not defined. Please ensure Cell 8 ('wH_9JspNo2Gz') has been executed.")
    answer, confidence, transparency, grounding = "Error: pipeline not defined. Run Cell 8.", "", "", ""
else:
    # Call pipeline with use_gemini=False to see direct embedding retrieval results
    answer, confidence, transparency, grounding = pipeline(question, top_k=3, doc_filter="All", use_gemini=False)

print("\n--- Question ---")
print(question)
print("\n--- Answer (from Embedding Retrieval) ---")
print(answer)
print("\n--- Confidence ---")
print(confidence)
print("\n--- Retrieval Transparency (Embeddings Results) ---")
print(transparency)
print("\n--- Grounding Prompt / Route ---")
print(grounding)

Running embedding search for the question...

--- Question ---
what is software testing strategies

--- Answer (from Embedding Retrieval) ---
I found the following information in the documents:

A strategy for software testing provides a road map that describes the steps to be conducted as part of testing, when these steps are planned and then undertaken, and how much effort, time, and resources will be required. Therefore, any testing strategy must incorporate test planning, test case design, test execution, and resultant data collection and evaluation. A software testing strategy should be flexible enough to promote a customized testing approach. At the same time, it must be rigid enough to encourage reason- able planning and management tracking as the project progresses. Shooman [Sho83] discusses these issues: In many ways, testing is an individualistic process, and the number of different types of tests varies as much as the different development approaches. For many years, our 449

In [ ]:
!pip -q install -U google-genai

import random, time
from google import genai
from google.genai import types
from google.colab import userdata

API_KEY = userdata.get("GOOGLE_API_KEY")
assert API_KEY, "Set GOOGLE_API_KEY in Colab Secrets"

client = genai.Client(
    api_key=API_KEY,
    http_options=types.HttpOptions(
        api_version="v1",
        timeout=90_000,  # milliseconds
        retry_options=types.HttpRetryOptions(
            attempts=4,          # total attempts including first try
            initial_delay=1.0,
            max_delay=15.0,
            # default retryable codes include 429 and 5xx in this SDK
        ),
    ),
)

current_question = globals().get("question")
assert current_question, "Run the embedding cell first so 'question' exists."

answer = globals().get("answer", "")
confidence = globals().get("confidence", "")
transparency = globals().get("transparency", "")
grounding = globals().get("grounding", "")

def clip(x, n=3500):
    x = "" if x is None else str(x)
    return x[:n] + ("…[trimmed]" if len(x) > n else "")

context = "\n\n".join([
    f"Answer:\n{clip(answer, 4000)}",
    f"Confidence:\n{clip(confidence, 500)}",
    f"Transparency:\n{clip(transparency, 4000)}",
    f"Route:\n{clip(grounding, 1200)}",
])

prompt = (
    "Use ONLY the context below to answer the query. If context is insufficient, say so.\n\n"
    f"Query: {current_question}\n\n"
    f"Context:\n{context}"
)

# streaming output
print("---- Gemini output ----")
out = []
for chunk in client.models.generate_content_stream(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=1024,
        temperature=0.2,
    ),
):
    if chunk.text:
        print(chunk.text, end="", flush=True)
        out.append(chunk.text)

print()
final_text = "".join(out)



---- Gemini output ----
A software testing strategy provides a roadmap that outlines the steps to be conducted as part of testing, including when these steps are planned and undertaken, and the required effort, time, and resources. It must incorporate test planning, test case design, test execution, and the collection and evaluation of resultant data.

A testing strategy should be flexible enough to allow for customized testing approaches, yet rigid enough to encourage reasonable planning and management tracking throughout the project. It addresses questions such as how to conduct tests, whether to develop a formal plan, if the entire program or only parts should be tested, when to rerun tests, and when to involve the customer. Essentially, it encompasses the approaches and philosophies for systematically planning and conducting testing activities.


# Task
Refactor the RAG pipeline by splitting the `pipeline` function in Cell 8 into a `pipeline_retrieve_and_route` function that solely handles retrieval and routing (returning raw results without LLM synthesis), remove `rag_answer`, and then update the `ui_query` function in Cell 11 to orchestrate the full RAG workflow: call `pipeline_retrieve_and_route` and conditionally use Gemini for synthesis. Additionally, modify the Gradio UI in Cell 11 to include a new textbox for `raw_retrieved_context_display` to show the raw context alongside the synthesized answer, ensuring all new return values are correctly mapped and displayed.

## Modify Retrieval and Routing Function

### Subtask:
Modify Cell 8 (`wH_9JspNo2Gz`). The `pipeline` function will be renamed to `pipeline_retrieve_and_route` and will be strictly responsible for retrieving raw document chunks or metadata answers. It will *not* call the Gemini model for synthesis. It will return `(retrieval_output, confidence, transparency, raw_context_for_llm, retrieval_route)`. The `rag_answer` function will be removed.


**Reasoning**:
The subtask requires modifying an existing code cell (`wH_9JspNo2Gz`) by renaming a function, removing another, and adjusting return values and a print statement as per the instructions. This single code block will apply all the requested changes to the cell.



In [ ]:
# Cell 8 — Pipeline functions (retrieve + rag + metadata routes)
# - Rebuilds index
# - Routes metadata queries (page count, list docs, chapters, author) before embeddings
# - If use_gemini=True: runs context-only RAG using google-genai (Fix 2)

# If you haven't installed yet:
# !pip -q install -U google-genai

from google import genai
from google.genai import types
from google.colab import userdata

# ----------------------------
# Gemini client (Fix 2) — cached
# ----------------------------
_GENAI_CLIENT = None

def get_genai_client():
    global _GENAI_CLIENT
    if _GENAI_CLIENT is not None:
        return _GENAI_CLIENT

    API_KEY = userdata.get("GOOGLE_API_KEY")
    assert API_KEY, "Set GOOGLE_API_KEY in Colab Secrets"

    _GENAI_CLIENT = genai.Client(
        api_key=API_KEY,
        http_options=types.HttpOptions(
            api_version="v1",
            timeout=90_000,  # ms
            retry_options=types.HttpRetryOptions(
                attempts=4,
                initial_delay=1.0,
                max_delay=15.0,
            ),
        ),
    )
    return _GENAI_CLIENT

def _clip(x, n=6000):
    x = "" if x is None else str(x)
    return x[:n] + ("…[trimmed]" if len(x) > n else "")

def rag_with_gemini_from_context(
    question: str,
    raw_context: str,
    transparency: str = "",
    model: str = "gemini-2.5-flash",
    max_output_tokens: int = 512,
    temperature: float = 0.2,
    stream: bool = False,
):
    """
    Context-only response: Gemini must use ONLY provided context.
    Uses timeout + retries configured on the client.
    """
    client = get_genai_client()

    # Keep context compact. Big PDF excerpts often cause slow/hangs.
    ctx = _clip(raw_context, 12000)
    trans = _clip(transparency, 4000)

    prompt = (
        "Answer the query using ONLY the context below.\n"
        "If the context does not contain the answer, say: 'Not found in the provided context.'\n\n"
        f"Query:\n{question}\n\n"
        f"Context:\n{ctx}\n\n"
        f"Retrieval Notes (optional):\n{trans}\n"
    )

    cfg = types.GenerateContentConfig(
        max_output_tokens=max_output_tokens,
        temperature=temperature,
    )

    if stream:
        out = []
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=prompt,
            config=cfg,
        ):
            if chunk.text:
                print(chunk.text, end="", flush=True)
                out.append(chunk.text)
        print()
        return "".join(out)
    else:
        resp = client.models.generate_content(
            model=model,
            contents=prompt,
            config=cfg,
        )
        return getattr(resp, "text", "") or ""

# ----------------------------
# Your existing index helpers
# ----------------------------
def rebuild_index(force=False):
    global pdf_paths, pages_df, chunks_df, emb_matrix, index, chapter_df, doc_stats
    pdf_paths = list_pdfs()
    if len(pdf_paths) == 0:
        return "No PDFs found. Upload files first."
    pages_df, doc_stats = extract_pdfs(pdf_paths)
    chunks_df = make_chunks(pages_df)
    chapter_df = extract_chapter_candidates(pages_df)
    if len(chunks_df) > MAX_CHUNKS:
        return f"Too many chunks ({len(chunks_df)}). Use fewer PDFs."
    chunks_df, emb_matrix, index = build_index(pdf_paths, force=force)
    if index:
        return f"Rebuilt index. PDFs: {len(pdf_paths)}, pages: {len(pages_df)}, chunks: {len(chunks_df)}, chapters: {len(chapter_df)}"
    return "Index rebuild failed."

def ensure_index_ready():
    if 'index' not in globals() or index is None or 'pages_df' not in globals() or pages_df is None:
        return rebuild_index(force=True)
    return "Index ready (cache loaded)."

def sync_pdfs():
    return rebuild_index(force=False)

# ----------------------------
# Your existing metadata routes
# ----------------------------
def chapter_answer(doc_filter="All"):
    if chapter_df is None or len(chapter_df) == 0:
        return "No chapter-like headings found.", "", "", "", "Chapter route"
    df = chapter_df if doc_filter == "All" else chapter_df[chapter_df.file_name == doc_filter]
    if len(df) == 0:
        return f"No chapter headings in {doc_filter}.", "", "", "", "Chapter route"
    lines = []
    for row in df.itertuples():
        cite = f"[{row.file_name} p.{row.page_number}]"
        lines.append(f"{cite} {row.heading}")
    answer_text = "Chapter-like headings I found:\n" + "\n".join(lines)
    transparency = "\n".join(lines)
    return answer_text, "Metadata", transparency, answer_text, "Chapter route"

def pagecount_answer(doc_filter="All"):
    if doc_stats is None or len(doc_stats) == 0:
        return "No PDFs indexed yet.", "", "", "", "Page-count route"
    df = doc_stats if doc_filter == "All" else doc_stats[doc_stats.file_name == doc_filter]
    if len(df) == 0:
        return f"No stats for {doc_filter}.", "", "", "", "Page-count route"
    lines = []
    for row in df.itertuples():
        lines.append(f"{row.file_name}: {row.total_pages} pages (low-text {row.low_text_pages})")
    ans = "\n".join(lines)
    return ans, "Metadata", ans, ans, "Page-count route"

def listdocs_answer():
    if doc_stats is None or len(doc_stats) == 0:
        return "No PDFs indexed yet.", "", "", "", "List docs route"
    lines = [f"{row.file_name} ({row.total_pages} pages)" for row in doc_stats.itertuples()]
    ans = "Available PDFs:\n" + "\n".join(lines)
    return ans, "Metadata", ans, ans, "List docs route"

def author_answer(doc_filter="All"):
    if doc_stats is None or len(doc_stats) == 0:
        return "No PDFs indexed yet.", "", "", "", "Author route"
    df = doc_stats if doc_filter == "All" else doc_stats[doc_stats.file_name == doc_filter]
    if len(df) == 0:
        return f"No author info for {doc_filter}.", "", "", "", "Author route"
    lines = []
    for row in df.itertuples():
        lines.append(f"{row.file_name}: {row.author} (Title: {row.title})")
    ans = "\n".join(lines)
    return ans, "Metadata", ans, ans, "Author route"

# ----------------------------
# Retrieval + route (unchanged logic)
# Returns 5-tuple: retrieval_output, conf, transparency, raw_context_for_llm, retrieval_route
# ----------------------------
def pipeline_retrieve_and_route(question, top_k=3, doc_filter="All"):
    if not question.strip():
        return "Please enter a question.", "", "", "", ""

    _ = ensure_index_ready()

    if is_list_docs_query(question):
        return listdocs_answer()
    if is_pagecount_query(question):
        return pagecount_answer(doc_filter)
    if is_author_query(question):
        return author_answer(doc_filter)
    if is_chapter_query(question):
        return chapter_answer(doc_filter)

    res = retrieve_chunks(question, k=int(top_k), doc_filter=doc_filter)
    if res["error"]:
        return res["error"], "", "", "", ""

    items = res["items"]
    top = items[0]
    conf = f"{confidence_label(top['score'])} (top score {top['score']:.3f})"

    raw_context_for_llm = "\n\n".join([item["chunk_text"] for item in items])
    retrieval_output = "I found the following information in the documents:\n\n" + raw_context_for_llm
    retrieval_route = "Direct embedding retrieval (no LLM)."

    lines = []
    for it in items:
        cite = f"[{it['file_name']} p.{it['page_number']}]"
        lines.append(f"Score: {it['score']:.3f} | {it['confidence']} | {cite}\n{it['preview']}")
    transparency = "\n\n".join(lines)

    return retrieval_output, conf, transparency, raw_context_for_llm, retrieval_route

# ----------------------------
# Public pipeline — matches your earlier usage:
# answer, confidence, transparency, grounding = pipeline(...)
# ----------------------------
def pipeline(question, top_k=3, doc_filter="All", use_gemini=True, stream=False,
             model="gemini-2.5-flash"):
    """
    Returns: (answer, confidence, transparency, grounding)
    - If metadata route: returns metadata answer (no Gemini).
    - If embedding route:
        - use_gemini=False => returns retrieval_output
        - use_gemini=True  => returns Gemini synthesized answer based ONLY on retrieved context
    """
    retrieval_output, conf, transparency, raw_ctx, route = pipeline_retrieve_and_route(
        question, top_k=top_k, doc_filter=doc_filter
    )

    # Metadata routes should not call Gemini (already a complete deterministic answer)
    if route in ("Chapter route", "Page-count route", "List docs route", "Author route"):
        return retrieval_output, conf, transparency, route

    # Errors
    if not conf and not transparency and not raw_ctx and route == "":
        return retrieval_output, conf, transparency, route

    if not use_gemini:
        return retrieval_output, conf, transparency, route

    # Gemini synthesis (context-only)
    try:
        llm_answer = rag_with_gemini_from_context(
            question=question,
            raw_context=raw_ctx,
            transparency=transparency,
            model=model,
            max_output_tokens=512,
            temperature=0.2,
            stream=stream,
        )
        grounding = "RAG with Gemini (context-only)."
        return llm_answer, conf, transparency, grounding
    except Exception as e:
        # Safe fallback: at least return retrieval output
        fallback = f"[Gemini error: {e}]\n\n{retrieval_output}"
        return fallback, conf, transparency, "Fallback to retrieval (Gemini failed)."

print("Use: pipeline(question, top_k=3, doc_filter='All', use_gemini=True/False, stream=False)")
print("Also available: sync_pdfs() / rebuild_index(force=True)")


Use: pipeline(question, top_k=3, doc_filter='All', use_gemini=True/False, stream=False)
Also available: sync_pdfs() / rebuild_index(force=True)


**Reasoning**:
The subtask requires defining the `ui_query` function to orchestrate the RAG workflow and then setting up the Gradio UI in Cell 11. This includes calling the `pipeline_retrieve_and_route` function, conditionally using the Gemini model for synthesis, and displaying the raw retrieved context alongside the synthesized answer in the Gradio interface. The `gradio` library is already imported as `gr` in a previous cell.



In [ ]:
import gradio as gr

# Cell 11 — Gradio UI (Fixed for google-genai / Fix 2)

# ---- Safety: make sure index is loaded so dropdown has choices ----
try:
    ensure_index_ready()
except Exception as e:
    print(f"Index not ready yet: {e}")

METADATA_ROUTES = {"Chapter route", "Page-count route", "List docs route", "Author route"}

def refresh_doc_choices():
    # pdf_paths may be list[Path]
    choices = ["All"]
    if "pdf_paths" in globals() and pdf_paths:
        choices += [p.name for p in pdf_paths]
    return choices

def rebuild_ui():
    msg = rebuild_index(force=True)
    return msg, gr.update(choices=refresh_doc_choices(), value="All")

def sync_ui():
    msg = sync_pdfs()
    return msg, gr.update(choices=refresh_doc_choices(), value="All")

def ui_query(question, top_k, doc_filter, use_gemini):
    if not str(question).strip():
        return "Please enter a question.", "", "", "", ""

    # Retrieve + route
    retrieval_output, confidence, transparency, raw_context_for_llm, retrieval_route = pipeline_retrieve_and_route(
        question, top_k, doc_filter
    )

    # Basic error handling (keep 5 outputs always)
    if not retrieval_route and (not confidence and not transparency and not raw_context_for_llm):
        # likely retrieval error returned in retrieval_output
        return retrieval_output, confidence, transparency, raw_context_for_llm, retrieval_route

    if isinstance(retrieval_output, str) and (
        retrieval_output.startswith("Error:") or
        retrieval_output.startswith("No PDFs found") or
        retrieval_output.startswith("Index rebuild failed")
    ):
        return retrieval_output, "", "", "", retrieval_route

    # Default: direct retrieval
    answer = retrieval_output
    grounding = retrieval_route

    # Skip Gemini on metadata routes (already deterministic)
    if retrieval_route in METADATA_ROUTES:
        grounding = f"{retrieval_route} (Gemini skipped for metadata)."
        return answer, confidence, transparency, raw_context_for_llm, grounding

    # Gemini synthesis (Fix 2) — ONLY if requested and we have context
    if use_gemini:
        if raw_context_for_llm and str(raw_context_for_llm).strip():
            try:
                answer = rag_with_gemini_from_context(
                    question=question,
                    raw_context=raw_context_for_llm,
                    transparency=transparency,
                    model="gemini-2.5-flash",
                    max_output_tokens=512,
                    temperature=0.2,
                    stream=False,   # set True if you want token streaming in notebook logs
                )
                grounding = "RAG with Gemini (context-only)."
            except Exception as e:
                answer = f"[Gemini error: {e}]\n\n{retrieval_output}"
                grounding = "Fallback to retrieval (Gemini failed)."
        else:
            grounding = "No context found (retrieval only)."

    return answer, confidence, transparency, raw_context_for_llm, grounding


# ---- Gradio Interface ----
with gr.Blocks(title="PDF RAG System") as demo:
    gr.Markdown("# PDF RAG with Gemini (google-genai) + FAISS")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Configuration")
            rebuild_btn = gr.Button("Rebuild Index (clears cache)")
            sync_btn = gr.Button("Sync PDFs (uses cache if unchanged)")
            status_output = gr.Textbox(label="Index Status", interactive=False)

            doc_filter_dropdown = gr.Dropdown(
                label="Filter by PDF Document",
                choices=refresh_doc_choices(),
                value="All",
                interactive=True,
                allow_custom_value=False,
            )

            top_k_slider = gr.Slider(minimum=1, maximum=10, value=3, step=1, label="Number of Chunks (k)")
            use_gemini_checkbox = gr.Checkbox(label="Use Gemini for Synthesis", value=True)

        with gr.Column(scale=2):
            gr.Markdown("## Query")
            question_input = gr.Textbox(label="Your Question", placeholder="e.g., What is software testing strategies?")
            submit_btn = gr.Button("Ask PDF")

            gr.Markdown("## Results")
            answer_output = gr.Textbox(label="Answer", interactive=False, lines=10)
            confidence_output = gr.Textbox(label="Confidence", interactive=False)
            grounding_output = gr.Textbox(label="Grounding / Route", interactive=False)
            raw_retrieved_context_display = gr.Textbox(label="Raw Retrieved Context", interactive=False, lines=10)
            transparency_output = gr.Textbox(label="Retrieval Transparency", interactive=False, lines=10)

    # Event Handlers (also refresh dropdown on rebuild/sync)
    rebuild_btn.click(rebuild_ui, outputs=[status_output, doc_filter_dropdown])
    sync_btn.click(sync_ui, outputs=[status_output, doc_filter_dropdown])

    submit_btn.click(
        ui_query,
        inputs=[question_input, top_k_slider, doc_filter_dropdown, use_gemini_checkbox],
        outputs=[answer_output, confidence_output, transparency_output, raw_retrieved_context_display, grounding_output]
    )
    question_input.submit(
        ui_query,
        inputs=[question_input, top_k_slider, doc_filter_dropdown, use_gemini_checkbox],
        outputs=[answer_output, confidence_output, transparency_output, raw_retrieved_context_display, grounding_output]
    )

# Optional: better for longer calls
demo.queue(default_concurrency_limit=1)

demo.launch(debug=True)



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://325fc87c8af807b1f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2157, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1634, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 63, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://325fc87c8af807b1f0.gradio.live
